In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os

ROOT_DIR = os.path.abspath('..')
sys.path.append(ROOT_DIR)
os.chdir(ROOT_DIR)

In [ ]:
from neuron.neurons import User, Neuron, ConversationalOrchestrator, ConversationalOrchestratorManager, UserTest
import os

In [ ]:
llm_config = {
    "config_list": [
        {
            "client": "groq",
            "temperature": 0.0,
            "model": "llama3-70b-8192",
            "api_key": os.getenv("GROQ_API_KEY")
        }
    ]
}

user = User(
    description="You are a human user interacting with conversational and recommendation agents."
)

conversational = Neuron(
    name="conversational",
    llm_config=llm_config,
    system_message="""
    You are a friendly Conversation AI.
Your job is to handle all general interactions naturally and pleasantly.

When a user asks for recommendations (such as requesting suggestions or asking for advice on choosing something), **you must immediately pass the request to the Recommender**.

- In case of a recommendation request, reply with: "Let me connect you with our Recommender to help with that!" and do not answer directly.
- Otherwise, continue interacting normally, answering questions, chatting, and helping the user in a casual, supportive tone.
- Always maintain a positive and welcoming attitude.

Focus on keeping the conversation flowing unless a specialist (like the Recommender) is needed.

""",
    description="This agent acts as a social companion and facilitates interactions."
)

llm_config_2 = {
    "config_list": [
        {
            "client": "groq",
            "temperature": 0.0,
            "model": "llama3-70b-8192",
            "api_key": os.getenv("GROQ_API_KEY")
        }
    ]
}

recommendation = Neuron(
    name="recommender",
    llm_config=llm_config_2,
    system_message="""
    You are a specialized Recommender AI.
Your role is to respond directly to user requests for recommendations.

When the user asks for recommendations (such as suggesting products, services, options, or anything similar), you must respond clearly and directly, providing a concise list or a short explanation.

- Never decline a recommendation request.
- Keep your responses focused only on recommending options.
- Do not ask questions unless necessary for a better recommendation.
- Stay friendly and helpful.

Examples of valid requests you should handle:
- "Can you recommend a good restaurant?"
- "Suggest a book to read."
- "What laptop should I buy?"

Always take the initiative to recommend something relevant, even if the user's question is vague.

""",
    description="This agent specializes in offering product recommendations."
)

chitchat = ConversationalOrchestrator(
    agents=[user, conversational, recommendation],
    messages=[],
    max_round=100,
    speaker_selection_method="auto",
    #send_introductions=True,
    allow_repeat_speaker = False,
)

chitchat_manager = ConversationalOrchestratorManager(chitchat=chitchat, llm_config=llm_config)

In [ ]:
chat_result = user.initiate_chat(
    chitchat_manager, message="Oi tudo bem?",
    cache=None
)

In [ ]:
user._oai_messages

In [ ]:
conversational._oai_messages